# PatchCore Example

#### Import dependencies

In [1]:
import os
import anodet
import numpy as np
import torch
import cv2
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# 

## Training

In this notebook the MVTec dataset will be used. It can be downloaded from: https://www.mvtec.com/company/research/datasets/mvtec-ad.

Change `DATASET_PATH` to the path where you save the dataset.

Also, if you want to save and load the model. Add a folder where to save them and change the path of `MODEL_DATA_PATH` to that folder.

In [2]:
DATASET_PATH = os.path.realpath("/home/wy/anodet/data/")
MODEL_DATA_PATH = os.path.realpath("/home/wy/anodet/distributions/")

#### Load dataset

In [3]:
dataset = anodet.AnodetDataset(os.path.join(DATASET_PATH, "bottle/train/good"))
dataloader = DataLoader(dataset, batch_size=32)
print("Number of images in dataset:", len(dataloader.dataset))

Number of images in dataset: 209


#### Init the model

In [4]:
patch_core = anodet.PatchCore()

/home/wy/anaconda3/envs/adet/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/wy/anaconda3/envs/adet/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


#### Fit the model to the dataset

In [ ]:
patch_core.fit(dataloader)

#### Save the necessary parameters

In [ ]:
torch.save(patch_core.embedding_coreset, os.path.join(MODEL_DATA_PATH, "bottle_embedding_coreset.pt"))

# 

## Inference
#### Load test images

In [ ]:
paths = [
    os.path.join(DATASET_PATH, "bottle/test/broken_large/000.png"),
    os.path.join(DATASET_PATH, "bottle/test/broken_small/000.png"),
    os.path.join(DATASET_PATH, "bottle/test/contamination/000.png"),
    os.path.join(DATASET_PATH, "bottle/test/good/000.png"),
    os.path.join(DATASET_PATH, "bottle/test/good/001.png"),
]

images = []
for path in paths:
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    images.append(image)
    
batch = anodet.to_batch(images, anodet.standard_image_transform, torch.device('cpu'))

#### Load the model data

In [ ]:
embedding_coreset = torch.load(os.path.join(MODEL_DATA_PATH, 'bottle_embedding_coreset.pt'))

#### init the model

In [ ]:
patch_core = anodet.PatchCore('resnet18', embedding_coreset=embedding_coreset)

#### Make prediction

In [ ]:
image_scores, score_maps = patch_core.predict(batch)

#### Interpret the prediction

In [ ]:
THRESH = 2.4
score_map_classifications = anodet.classification(score_maps, THRESH)
image_classifications = anodet.classification(image_scores, THRESH)
print("Image scores:", image_scores)
print("Image classifications:", image_classifications)

#### Visualization

In [ ]:
test_images = np.array(images).copy()

In [ ]:
!python -V

In [ ]:
boundary_images = anodet.visualization.framed_boundary_images(test_images, score_map_classifications, image_classifications, padding=40)
heatmap_images = anodet.visualization.heatmap_images(test_images, score_maps, alpha=0.5)
highlighted_images = anodet.visualization.highlighted_images(images, score_map_classifications, color=(128, 0, 128))

for idx in range(len(images)):
    fig, axs = plt.subplots(1, 4, figsize=(12, 6))
    fig.suptitle('Image: ' + str(idx), y=0.75, fontsize=14)
    axs[0].imshow(images[idx])
    axs[1].imshow(boundary_images[idx])
    axs[2].imshow(heatmap_images[idx])
    axs[3].imshow(highlighted_images[idx])
    plt.savefig(f'result_{idx}.png')
    plt.show()
    

#### For one merged image

In [ ]:
heatmap_images = anodet.visualization.heatmap_images(test_images, score_maps, alpha=0.5)
tot_img = anodet.visualization.merge_images(heatmap_images, margin=40)
fig, axs = plt.subplots(1, 1, figsize=(10, 6))
plt.imshow(tot_img)
plt.savefig('heatmap.png')
plt.show()